<img src="https://electronaut.info/wp-content/uploads/2017/05/Tubescape_jupyter2.jpg">

<h1><font color = b30000>TUBE DATA MATCHER for uTRACER FILES</font></h1><h5>Anode current for a sweep of grid voltages while anode voltage is constant</h5>


# STEP 1 — Import and process tube data, build dataframe.

In [1]:
from ElectronauTracer_Importer_restruct import *

Please enter the path to the folder containing the uTracer files: /Users/robroycampbell/Documents/Electronaut Co/Tube Measurements/5749 Vari-Mu Pentodes/5749_Stock_Burned In/5749 measurements Jan 2020

What type of tubes are these? 5749

— — — — — 

Data from 237 tubes of type 5749 was successfully read.
A dataframe containing all 237 tubes was successfully created.
Stastistics calculated.

Starting difference calculations...  (** This can take a while **)


FloatProgress(value=0.0)

It took 24.63 seconds to calculate 898704 values, resulting in 56169 rows.


### Start the Matched Set Finder

In [2]:


# STEP 7:  Build a dataframe of the best matched sets in order

# math lib used for generating ordinal strings; i.e. ['1st', '2nd', '3rd'] etc.
import math

# Make a local version of df_dif that can be butchered
df_dif_copy = df_dif.copy()

def ask_tube_set_size():
    
    # Ask the user to enter the set size
    tube_set_size = input("Please enter the number of tubes per matched set:")
    
    # Build a list of tube names for the tube set size
    tube_sets_columns = ['Tube_' + str(1 + i) for i in range(int(tube_set_size))]    
    
    return tube_set_size, tube_sets_columns 


def find_best_set_for_chosen_tube_ID(df_dif_copy, tube_to_match, tube_set_size, tube_sets_columns):
    tube_to_match = str(tube_to_match)
        
    # Create a new dataframe showing only tube_set_size number of nearest matches to the chosen tube_to_match
    df_tube_set = df_dif_copy[(df_dif_copy.ref_tube_ID == tube_to_match)].sort_values('error_sum').head(int(tube_set_size))
        
    # Create the dictionary of the tubes in the set and the total score of the set
    tube_set_list = list(df_tube_set.match_tube_ID)
    tube_set_dict = dict(zip(tube_sets_columns, tube_set_list))   
    tube_set_score = df_tube_set.error_sum.sum()
    tube_set_dict.update({'tube_set_score':tube_set_score})

    return tube_set_dict


def find_best_set(df_dif_copy, tube_set_size, tube_sets_columns):
    
    # Make an empty list to hold the tube sets and their scores
    tube_sets_and_scores_list = []
    temp_dict = {}
    
    if 'tube_set_score' in tube_sets_columns:
        tube_sets_columns.remove('tube_set_score')

    for i in df_dif_copy.ref_tube_ID.unique():
        temp_dict = find_best_set_for_chosen_tube_ID(df_dif_copy, i, tube_set_size, tube_sets_columns)
        tube_sets_and_scores_list.append(temp_dict)

    # Add 'tube_set_score' column name at the front of the list
    if 'tube_set_score' not in tube_sets_columns: 
        tube_sets_columns.insert(0, 'tube_set_score')    
    
    # Create a dataframe for the tube sets called df_tube_sets 
    df_tube_sets = pd.DataFrame(tube_sets_and_scores_list, columns=tube_sets_columns)
    
    # Sort the tube_set_score column, take the top row (with the lowest score) as a new df 
    df_best_set_temp = df_tube_sets.sort_values('tube_set_score').head(1)
        
    return df_best_set_temp


# A function to plot a line graph of a set of tubes by passing in a list of tube numbers
def plot_tubes(list_of_tubes, df):
    
    output_file("Tube_Set_Statistics.html")

    # Configure the size, title, etc.
    p = figure(plot_width=1000, plot_height=750, title=str(list_of_tubes))
    p.title.text_color = "black"

    # Make a list for the x_values by chopping off all the "bias_" column name prefixes and converting to ints
    temp_columns = list(df.columns)
    x_values = [int(i[5:]) for i in temp_columns if i.startswith('Bias_') == True]        


    # Set axis labels
    p.xaxis.axis_label = "Grid Voltage (V)"
    p.xaxis.axis_label_text_color = "#aa6666"
    p.yaxis.axis_label = "Plate Current (mA)"

    # Set grid lines
    p.ygrid.minor_grid_line_color = 'navy'
    p.ygrid.minor_grid_line_alpha = 0.2
    p.xgrid.minor_grid_line_color = 'navy'
    p.xgrid.minor_grid_line_alpha = 0.2

    # Set background color
    p.background_fill_color = "beige"
    p.background_fill_alpha = 0.5

    for tube in list_of_tubes:
        row = ((df.loc[df['tube_ID'] == tube[0:3]]).values.tolist())[0]
        row = [float(i) for i in row[3:]]
        p.line(x_values, row, line_width=2)

    return p    


    
# Start a timer
start_time = time.time()   
    
# Make an empty dataframe to hold all the sets and their scores
df_best_tube_sets = pd.DataFrame()

# Get the tube_set_size and the tube_sets_columns from the ask_tube_set_size function
tube_set_size, tube_sets_columns = ask_tube_set_size()

# Get the best tube set from the batch
df_best_set_temp = find_best_set(df_dif_copy, tube_set_size, tube_sets_columns) 

# Display a progress bar
progress_bar_range = int(len(df)/int(tube_set_size))
best_matched_progress_bar = FloatProgress(min=0, max=progress_bar_range)
display(best_matched_progress_bar)

# Append the best tube set to the df_best_tube_sets dataframe
df_best_tube_sets = df_best_tube_sets.append(df_best_set_temp)

# Remove the best tube set from df_dif_copy
tube_delete_list = list(df_best_set_temp.iloc[0])
tube_delete_list.pop(0)
df_dif_copy = df_dif_copy[~df_dif_copy.ref_tube_ID.isin(tube_delete_list)]
df_dif_copy = df_dif_copy[~df_dif_copy.match_tube_ID.isin(tube_delete_list)]

# A function to generate ordinal strings; i.e. ['1st', '2nd', '3rd'] etc.
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(math.floor(n/10)%10!=1)*(n%10<4)*n%10::4])
ordinal_list = [ordinal(n) for n in range(1, int(len(df)))]

set_count = 1
print()
print("*** BEST! ***")
print(ordinal_list[0],"best tube set SCORE:", str(df_best_set_temp.tube_set_score.iloc[0]))
print("TUBES:", tube_delete_list)

# Update the progress bar
best_matched_progress_bar.value += 1

tube_matches_master_plot_list = []

while len(df_dif_copy.ref_tube_ID.unique()) >= int(tube_set_size):

    # Get all the rest of the sets from the batch
    df_best_set_temp = find_best_set(df_dif_copy, tube_set_size, tube_sets_columns)
    
    # Append the best tube set to the df_best_tube_sets dataframe
    df_best_tube_sets = df_best_tube_sets.append(df_best_set_temp)

    # Remove the best tube set from df_dif_copy
    tube_delete_list = list(df_best_set_temp.iloc[0])
    tube_delete_list.pop(0)

    df_dif_copy = df_dif_copy[~df_dif_copy.ref_tube_ID.isin(tube_delete_list)]
    df_dif_copy = df_dif_copy[~df_dif_copy.match_tube_ID.isin(tube_delete_list)]
    print()
    print(ordinal_list[set_count],"best tube set SCORE:", str(df_best_set_temp.tube_set_score.iloc[0]))
    print("TUBES:", tube_delete_list)
    tube_delete_list.insert(0, ordinal)

    tube_matches_master_plot_list.insert(1 ,tube_delete_list[1:])

    set_count += 1
    
    # Update the progress bar
    best_matched_progress_bar.value += 1

    
    



stop_time = time.time()
elapsed_time = stop_time - start_time
print()
print("It took", round(elapsed_time, 2), "seconds to produce", set_count, "tube sets.")

# print(tube_matches_master_plot_list[0])

# plot_tubes((list(df_best_tube_sets.iloc[0])[1:]), df)
    
# Build the plot object
# tube_plot = plot_tubes(tube_delete_list, df)

# Show the plot
# show(tube_plot)



best_set_list = list(df_best_tube_sets.iloc[0])[1:]
worst_set = df_best_tube_sets.tail(1)
worst_set_list = list(worst_set.iloc[0])[1:]






Please enter the number of tubes per matched set:16


FloatProgress(value=0.0, max=14.0)


*** BEST! ***
1st best tube set SCORE: 38.79435300000003
TUBES: ['655', '597', '440', '663', '727', '654', '425', '601', '678', '416', '777', '738', '791', '775', '662', '717']

2nd best tube set SCORE: 45.43804299999998
TUBES: ['584', '622', '596', '575', '576', '590', '619', '599', '496', '585', '651', '567', '538', '549', '586', '572']

3rd best tube set SCORE: 46.14140000000006
TUBES: ['702', '472', '605', '706', '721', '594', '680', '550', '673', '595', '708', '666', '603', '729', '540', '405']

4th best tube set SCORE: 47.81206200000008
TUBES: ['690', '725', '684', '607', '454', '401', '361', '407', '737', '404', '661', '668', '448', '646', '628', '788']

5th best tube set SCORE: 55.28115600000004
TUBES: ['644', '547', '608', '534', '477', '548', '560', '669', '563', '491', '421', '639', '433', '683', '699', '362']

6th best tube set SCORE: 66.10696799999994
TUBES: ['552', '532', '724', '418', '611', '475', '609', '677', '716', '645', '688', '591', '410', '714', '718', '364']

7

# MATCHED SETS (lower score is better)

In [315]:
df_best_tube_sets.head()

,tube_set_score,Tube_1,Tube_2,Tube_3,Tube_4,Tube_5,Tube_6,Tube_7,Tube_8,Tube_9,Tube_10,Tube_11,Tube_12,Tube_13,Tube_14,Tube_15,Tube_16
42,38.794353,655,597,440,663,727,654,425,601,678,416,777,738,791,775,662,717
61,45.438043,584,622,596,575,576,590,619,599,496,585,651,567,538,549,586,572
197,46.141400,702,472,605,706,721,594,680,550,673,595,708,666,603,729,540,405
9,47.812062,690,725,684,607,454,401,361,407,737,404,661,668,448,646,628,788
15,55.281156,644,547,608,534,477,548,560,669,563,491,421,639,433,683,699,362


# Plot any tube set by entering the row number 

In [4]:
# PLOT by row number
row_number = 10

# Make the list of tubes to plot
tubes_to_plot = df_best_tube_sets.iloc[row_number].tolist()[1:]

# Build the plot object
tube_plot = plot_tubes(tubes_to_plot, df)

show(tube_plot)

# Find all possible complimentary pairs of 4-tube sets from an 8-tube list

In [285]:
tubelist = tubes_to_plot.copy()[0:8]

# make a temoporary working dataframe containing only the tubes in the tubelist
tubelist


['439', '495', '697', '615', '629', '499', '369', '497']

In [289]:
tubelist_df = df[df['tube_ID'].isin(tubelist)]
tubelist_df

,anode_voltage,tube_type,tube_ID,Bias_2,Bias_6,Bias_10,Bias_14,Bias_18,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50
33,325,5749,697,92.14,57.108,37.019,27.265,20.761,15.949,12.378,9.704,7.701,6.242,5.195,4.426,3.854
37,325,5749,495,91.987,57.21,37.561,27.754,21.076,16.176,12.571,9.907,7.932,6.462,5.368,4.517,3.858
57,325,5749,497,90.359,55.48,36.035,26.506,20.079,15.4,11.94,9.361,7.478,6.13,5.149,4.352,3.694
73,325,5749,369,90.207,54.989,35.866,26.94,20.91,16.403,13.032,10.487,8.578,7.178,6.15,5.368,4.768
146,325,5749,439,91.377,56.566,36.93,27.347,20.927,16.169,12.649,10.014,8.071,6.659,5.624,4.834,4.217
175,325,5749,629,92.767,56.379,36.666,27.042,20.598,15.915,12.475,9.918,8.036,6.672,5.658,4.839,4.175
177,325,5749,615,90.851,55.82,36.625,27.354,21.161,16.508,13.019,10.328,8.207,6.557,5.304,4.373,3.685
204,325,5749,499,91.02,56.379,36.958,27.313,20.815,15.946,12.319,9.592,7.545,6.043,4.906,3.997,3.294


In [290]:
# Make a list of all possible 4-tube sets from a list of 8 tubes
from itertools import combinations

def find_all_possible_combinations(tubelist):
    all_possible_combinations = list(combinations(tubelist, 4))
    return all_possible_combinations

# Find the compliment to any 4-tube set
def find_compliment(combination, all_possible_combinations):
    filtered_list = all_possible_combinations.copy()
    for each_tube in combination:
        filtered_list = list(filter(lambda x: each_tube not in x, filtered_list)) 
    compliment = filtered_list[0]    
    return compliment

# Compile a master list of all the possible 4-tube combinations
def make_list_of_pairs(tubelist):
    all_possible_combinations = find_all_possible_combinations(tubelist)
    temp_combinations_list = all_possible_combinations.copy()
    list_of_pairs = []
    for each_combination in all_possible_combinations:
        if each_combination in temp_combinations_list:
            temp_compliment = find_compliment(each_combination, all_possible_combinations)
            temp_combinations_list.remove(temp_compliment)
            temp_combinations_list.remove(each_combination)
            temp_combo_pair = (each_combination, temp_compliment)
            list_of_pairs.append(temp_combo_pair)
    return list_of_pairs

all_possible_pairs = make_list_of_pairs(tubelist)

In [314]:
# Mechanism for totalling up all the currents for combinations and complimentary pairs

# A function for looking up the three tube currents of interest by passing the tube number
def get_currents(tube_ID, tubelist_df):
    bias_2_current = float(tubelist_df.loc[tubelist_df['tube_ID'] == tube_ID, 'Bias_2'].iloc[0])
    bias_22_current = float(tubelist_df.loc[tubelist_df['tube_ID'] == tube_ID, 'Bias_22'].iloc[0])
    bias_50_current = float(tubelist_df.loc[tubelist_df['tube_ID'] == tube_ID, 'Bias_50'].iloc[0])
    return (bias_2_current, bias_22_current, bias_50_current)

# A function for totalling up the currents of combinations and complimentary pairs
# Returns a dictionary

def total_up_currents(combination_pair):
    combination_bias_2 = 0.0
    combination_bias_22 = 0.0
    combination_bias_50 = 0.0
    compliment_bias_2 = 0.0
    compliment_bias_22 = 0.0
    compliment_bias_50 = 0.0    
    combination = combination_pair[0]
    compliment = combination_pair[1]
    
    for tube in combination:
        combination_currents_tuple = get_currents(tube, tubelist_df)
        combination_bias_2 = combination_bias_2 + combination_currents_tuple[0]
        combination_bias_22 = combination_bias_22 + combination_currents_tuple[1]
        combination_bias_50 = combination_bias_50 + combination_currents_tuple[2]
    for tube in compliment:
        compliment_currents_tuple = get_currents(tube, tubelist_df)
        compliment_bias_2 = compliment_bias_2 + compliment_currents_tuple[0]
        compliment_bias_22 = compliment_bias_22 + compliment_currents_tuple[1]
        compliment_bias_50 = compliment_bias_50 + compliment_currents_tuple[2]

    combo_pair_dict = {'combination' : combination, 'combination_bias_2' : combination_bias_2, 
                       'combination_bias_22' : combination_bias_22, 'combination_bias_50' : combination_bias_50, 
                      'compliment' : compliment, 'compliment_bias_2' : compliment_bias_2, 'compliment_bias_22' : compliment_bias_22,
                      'compliment_bias_50': compliment_bias_50}                          
                              
    return combo_pair_dict



# A function to iterate through the all_possiuble_pairs list and total up all the currents
# Should make a master list of dictionaries.

def total_up_everything(all_possible_pairs, tubelist_df):
    list_of_combination_dicts = []
    for combination_pair in all_possible_pairs:
        temp_dict = total_up_currents(combination_pair)
        list_of_combination_dicts.append(temp_dict)
    return list_of_combination_dicts

list_of_combination_dicts = total_up_everything(all_possible_pairs, tubelist_df)
combinations_df = pd.DataFrame(list_of_combination_dicts)
combinations_df.head()

,combination,combination_bias_2,combination_bias_22,combination_bias_50,compliment,compliment_bias_2,compliment_bias_22,compliment_bias_50
0,"(439, 495, 697, 615)",366.355,64.802,15.614,"(629, 499, 369, 497)",364.353,63.664,15.931
1,"(439, 495, 697, 629)",368.271,64.209,16.104,"(615, 499, 369, 497)",362.437,64.257,15.441
2,"(439, 495, 697, 499)",366.524,64.240,15.223,"(615, 629, 369, 497)",364.184,64.226,16.322
3,"(439, 495, 697, 369)",365.711,64.697,16.697,"(615, 629, 499, 497)",364.997,63.769,14.848
4,"(439, 495, 697, 497)",365.863,63.694,15.623,"(615, 629, 499, 369)",364.845,64.772,15.922


# STEP 6: Find the best matches for a particular tube_ID


In [ ]:
def find_best_set_for_chosen_tube_ID():
    
    tube_to_match = input("What tube number would you like to match? ")
    tube_set_size = int(input("How many matches would you like? ")) + 1
    
    # Create a new dataframe showing only tube_set_size number of nearest matches to the chosen tube_to_match
    df_tube_set = df_dif[(df_dif.ref_tube_ID == str(tube_to_match))].sort_values('error_sum').head(tube_set_size)
    
    return df_tube_set

df_tube_set = find_best_set_for_chosen_tube_ID()
df_tube_set.head(int(tube_set_size))

# STEP 4: Show the distribution of a single bias voltage


In [ ]:
# Histogram of a single column

import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import numpy as np

sigma = df_stats.iat[2, 5]
mu = df_stats.iat[1, 5]
x = mu + sigma  * np.random.randn(437)
num_bins = 50

fig, ax = plt.subplots()

# the histogram of the data
n, bins, patches = ax.hist(x, num_bins, normed=1)

# add a 'best fit' line
y = mlab.normpdf(bins, mu, sigma)
ax.plot(bins, y, '--')
ax.set_xlabel('Current (mA)')
# ax.set_ylabel('Probability density')
ax.set_title(r'Histogram of -10V grid voltage')

# Tweak spacing to prevent clipping of ylabel
fig.tight_layout()
plt.show()

## Compute the total currents of every possible combination of quads

#### Prepare the data

In [ ]:
import itertools

# make a reference copy of the main df and set the index of the main df to be the tube_ID number
df_copy = df.copy()
df_copy.set_index('tube_ID', inplace=True)

# make a df containing only the best set
best_set_df = df_copy.loc[best_set_list]

# make a list of the bias voltages
bias_list = best_set_df.columns.tolist()[2:]

#### Make a list of all possible sets of 4

In [ ]:
# Make a list of all possible sets of 4 from the best_set list
combinations_list = []
for combination in itertools.combinations(best_set_list, 4):
    combinations_list.append(combination)


#### General function definitions

In [ ]:
# Function to extract the current from a specified tube and bias voltage
def get_current(tube, bias):
    
    # Get the current from the tube at the specified bias (returned as dataframe)
    buzzz = best_set_df.loc[[tube], [bias]]
    
    # Convert to dict (results in nested dict)
    buzz_dict = buzzz.to_dict()
    # Looks like:  {'Bias_x' : {tube_ID : current}}
    
    # Get the name of the first dict key
    buzz_key = str(list(buzz_dict.keys())[0])

    # Extract the nested dict
    buzz_value_dict = buzz_dict[buzz_key]

    # Get the name of the nested dict key
    buzz_value_key = str(list(buzz_value_dict.keys())[0])

    # extract the value and turn it into a float
    buzz = float(buzz_value_dict[buzz_value_key])
    
    return buzz

# Function to total up the currents for a given combo tuple and bias voltage
def add_currents(combo_tuple, bias):
    total_current = 0.0
    for eachtube in combo_tuple:
        zzz = get_current(eachtube, bias)
        total_current += zzz
    return {bias : total_current}

# Function to build a dictionary for each combination containing total currents from each bias voltage
def build_combination_data(eachcombo, bias_list):
    combo_big_dict = {}
    for eachbias in bias_list:
            bias_dict = add_currents(eachcombo, eachbias)
            combo_big_dict.update(bias_dict)
            temp_dict = {'Combination' : eachcombo}
            combo_big_dict.update(temp_dict)
    return combo_big_dict

# Function for building a new list including only valid complimentary quads
def trim_combo_list(combination_tuple, combinations_list):
    trimmed_combo_list = [i for i in combinations_list if (combination_tuple[0]) not in i]
    trimmed_combo_list = [i for i in trimmed_combo_list if (combination_tuple[1]) not in i]
    trimmed_combo_list = [i for i in trimmed_combo_list if (combination_tuple[2]) not in i]
    trimmed_combo_list = [i for i in trimmed_combo_list if (combination_tuple[3]) not in i]
    print(len(trimmed_combo_list), "valid complimentary quads.")
    return trimmed_combo_list


In [ ]:
Rob Roy# Hard-coding an example
combination_tuple = combinations_list[0]

trimmed_combinations_list = trim_combo_list(combination_tuple, combinations_list)

print(len(trimmed_combinations_list), "valid combinations")

# trimmed_combination_list

#### build a dataframe of combinations and the total currents for each bias position

In [ ]:
# Build a dataframe showing the summed current for every tube and bias combination
# Prepare the master list for the dataframe

# Stopwatch
start_combos_time = time.time()


combos_data_list = []
for eachcombo in trimmed_combinations_list:
    thing = build_combination_data(eachcombo, bias_list)
    combos_data_list.append(thing)

    # Make a list of column names for the dataframe    
after_for_loop = time.time()
combo_columns = ['Combination'] + bias_list

# Build the dataframe
combo_df = pd.DataFrame(combos_data_list, columns=combo_columns)
after_df_generation = time.time()
print("For-loop took", (after_for_loop - start_combos_time), 'seconds.')
print("Dataframe generation took", (after_df_generation - after_for_loop), 'seconds.')
print("Total elapsed time:", (after_df_generation-start_combos_time))
print(len(combo_df), 'total possible combinations')
combo_df.head()    

#### Make a temp copy of combo_df and include only rows from the trimmed_combinations_list 

In [ ]:
Rob Roy# Make a temp copy of combo_df and include only rows from the trimmed_combinations_list 
temp_df2 = combo_df
temp_df2[temp_df2['Combination'].isin(trimmed_combinations_list)]

In [ ]:
# Sandbox

def mask(df, key, value):
    return df[df[key] == value]

pd.DataFrame.mask = mask

thing = combo_df['Combination'][0]

# combo_df.mask('A', 1).mask('D', 6)
temp_df = combo_df.mask('Combination', thing)
temp_df

In [ ]:
# Single-value bias matcher.

# TODO: Get the target current.  What is being matched?



def single_value_matcher(combo_df, combination_tuple, bias):
    combo_df = combo_df.sort_values([bias])
    


In [ ]:
# Build a function to mask the df by combination_tuple

def mask(df, key, value):
    return df[df[key] == value]

pd.DataFrame.mask = mask
combination_tuple_b = trimmed_combinations_list[3]

# combo_df.mask('A', 1).mask('D', 6)
temp_df = combo_df.mask('Combination', combination_tuple_b)
temp_df

In [ ]:
# # Make a simple matcher to find the closest matched currents at a single bias voltage

# test_bias = bias_list[0]

# totalcombos = combo_df.index.tolist()
# testcombo = totalcombos[0]
    
# def find_closest_combo(combo_df, bias):
    
#     # Sort the dataframe by the chosen bias column
#     combo_df = combo_df.sort_values([bias])

#     # Pick the combo to match (in this case, a fixed sample instead of an iterated loop)
#     totalcombos = combo_df.index.tolist()
#     testcombo = totalcombos[0]
#     nextcombo = totalcombos[1]
#     previouscombo = totalcombos[-1]
    
#     # Get the reference value
#     ref_value = combo_df.loc[testcombo, bias]

#     # Get the previous and next values to compare
#     next_value = combo_df.loc[nextcombo, bias]
#     previous_value = combo_df.loc[previouscombo, bias]
    
#     # Figure out which one is closer
#     diff_next = abs(float(ref_value) - float(next_value))
#     diff_previous = abs(float(ref_value) - float(previous_value))
    
#     # Return the closer match
#     if diff_next < diff_previous:
#         matchcombo = combo_df.loc[nextcombo, 'Combination']
#     else:
#         matchcombo = combo_df.loc[previouscombo, 'Combination']
#     return matchcombo

# matchedcombo = find_closest_combo(combo_df, test_bias)
# refcombo = combo_df.loc[testcombo, 'Combination']

# print("Refcombo:", refcombo, "  Current at bias voltage:", (combo_df.loc[testcombo, test_bias]))
# print("Matchcombo:", matchedcombo, "  Current at bias voltage:", (combo_df.loc[matchedcombo, test_bias]))



# # test_df = find_closest_combo(combo_df, test_bias)
# # test_df

# STEP 8: Plot a line graph of the best set


In [ ]:
# Plot a line graph of the best set

from bokeh.plotting import figure, output_file, show

output_file("Best Matched Tube Set.html")

# Configure the size, title, etc.
p = figure(plot_width=1000, plot_height=750, title="Plate Current as a function of Grid Voltage")
p.title.text_color = "black"

# Make a list for the x_values by chopping off all the "bias_" column name prefixes and converting to ints
temp_columns = list(df.columns)
x_values = [int(i[5:]) for i in temp_columns if i.startswith('Bias_') == True]        

for i in range(len(best_set)):
    row = df[df['tube_ID'] == best_set[i]]
    y_values = row.values.tolist()[0][3:]
    p.line(x_values, y_values, line_width=2)

# Set axis labels
p.xaxis.axis_label = "Grid Voltage (V)"
p.xaxis.axis_label_text_color = "#aa6666"
p.yaxis.axis_label = "Plate Current (mA)"

# Set grid lines
p.ygrid.minor_grid_line_color = 'navy'
p.ygrid.minor_grid_line_alpha = 0.2
p.xgrid.minor_grid_line_color = 'navy'
p.xgrid.minor_grid_line_alpha = 0.2

# Set background color
p.background_fill_color = "beige"
p.background_fill_alpha = 0.5

# Show the plot
show(p)

# STEP 9: Plot a line graph of the worst set


In [ ]:
Rob Roy M Campbell# Plot a line graph of the worst set

from bokeh.plotting import figure, output_file, show

output_file("Worst Matched Tube Set.html")

# Configure the size, title, etc.
p = figure(plot_width=1000, plot_height=750, title="Plate Current as a function of Grid Voltage")
p.title.text_color = "black"

# Make a list for the x_values by chopping off all the "bias_" column name prefixes and converting to ints
temp_columns = list(df.columns)
x_values = [int(i[5:]) for i in temp_columns if i.startswith('Bias_') == True]        

for i in range(len(worst_set)):
    row = df[df['tube_ID'] == worst_set[i]]
    y_values = row.values.tolist()[0][3:]
    p.line(x_values, y_values, line_width=2)

# Set axis labels
p.xaxis.axis_label = "Grid Voltage (V)"
p.xaxis.axis_label_text_color = "#aa6666"
p.yaxis.axis_label = "Plate Current (mA)"

# Set grid lines
p.ygrid.minor_grid_line_color = 'navy'
p.ygrid.minor_grid_line_alpha = 0.2
p.xgrid.minor_grid_line_color = 'navy'
p.xgrid.minor_grid_line_alpha = 0.2

# Set background color
p.background_fill_color = "beige"
p.background_fill_alpha = 0.5

# Show the plot
show(p)

### Everything below this line is experimental

In [ ]:
Rob Roy M Campbelldf_best_tube_sets.tube_set_score[:15].describe()


In [ ]:
import seaborn as sns
sns.set()

df_sns = df_best_tube_sets[['tube_set_score']][:15].copy()

sns.pairplot(df_sns, 'tube_set_score')
sns.plt.show()


In [ ]:
from bokeh.charts import Histogram, output_file, show

p = Histogram(df_best_tube_sets['tube_set_score'], title="Tube Set Scores")

output_file("histogram.html",)

show(p)

# Other Miscellaneous stuff

##### Apply a function to a column
df.Bias_2.max()

In [ ]:
% lsmagic

In [ ]:
%pprint



In [ ]:
# Applymap is useful for applying a function to all data
# For example, converting all data from floats to ints
# df.loc[2:7, 'Bias_2': 'Bias_50'].applymap(int)

In [ ]:
# List all available magics
% lsmagic

In [ ]:
# Display environmental variables
env_variables = %env
current_path = env_variables.get('PATH')
current_path

In [ ]:
mydict = % env
mydict.keys()

In [ ]:
% env

In [ ]:
# % time x = range(10000)
% timeit x = range(10000)
max(x)